<a href="https://colab.research.google.com/github/malloyca/CSC581B/blob/main/Final%20Project/vgg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CSC581B - Deep Learning for Image Classification

# VGG-16 Implementation

In [59]:
# imports
import torch
from torch import nn
from torch.utils.data import DataLoader, SubsetRandomSampler
from torchvision.datasets import CIFAR100
from torchvision import transforms
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import copy
import numpy as np
import random


In [60]:
expansion_coef = 1

In [61]:
data_augmentation_transform = transforms.Compose([
  transforms.RandomHorizontalFlip(p=0.5),
  transforms.RandomRotation(degrees=30),
  transforms.RandomCrop(32, padding=2),
  transforms.ColorJitter(brightness=0.25, contrast=0.5, saturation=0.25, hue = 0.15),
  transforms.RandomGrayscale(p=0.2),
  transforms.ToTensor(),
  transforms.Normalize(
      mean = [0.5071, 0.4867, 0.4408],
      std = [0.2675, 0.2565, 0.2761]
  ),
  transforms.RandomErasing(p=.35),
])

In [62]:
normalize_transform = transforms.Compose([
  transforms.Resize(size=32 * expansion_coef),
  transforms.ToTensor(),
  transforms.Normalize(
      mean = [0.5071, 0.4867, 0.4408],
      std = [0.2675, 0.2565, 0.2761]
  ),
])

## Load CIFAR-100

Load the dataset and split into training and test sets.

In [63]:
# Load the training data (CIFAR10 to start)
training_data = CIFAR100(
    root = "data",
    train = True,
    download = True,
    #transform = data_augmentation_transform,
    transform = normalize_transform,
)

# this is necessary to prevent the data augmentation transforms from being applied to the validation set
validation_data = CIFAR100(
    root = "data",
    train = True,
    download = True,
    transform = normalize_transform,
)

# Load the test data
test_data = CIFAR100(
    root = "data",
    train = False,
    download = True,
    transform = transforms.ToTensor()
)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [64]:
training_targets = training_data.targets

In [65]:
train_split_index, valid_split_index = train_test_split(
    np.arange(len(training_targets)), test_size=0.2, stratify=training_targets
)

In [83]:
batch_size = 100

# Create data loaders
train_dataloader = DataLoader(training_data, batch_size=batch_size,
                              sampler=SubsetRandomSampler(train_split_index))
valid_dataloader = DataLoader(validation_data, batch_size=batch_size,
                              sampler=SubsetRandomSampler(valid_split_index))
test_dataloader = DataLoader(test_data, batch_size=batch_size)

In [84]:
# Check that it is splitting the data properly
train_length = 0
for _, y in train_dataloader:
  train_length += len(y)
print(f"Length of training split: {train_length}")

valid_length = 0
for _, y in valid_dataloader:
  valid_length += len(y)
print(f"Length of validation split: {valid_length}")

test_length = 0
for _, y in test_dataloader:
  test_length += len(y)
print(f"Length of test split: {test_length}")

Length of training split: 40000
Length of validation split: 10000
Length of test split: 10000


In [85]:
# Check that there are 100 instances of a random class in the validation set
count = 0
test_class = random.randint(0,99)
for _, y in valid_dataloader:
  for target in y:
    if int(target.numpy()) == test_class:
      count += 1

print(count)

100


# Building VGG-16

In [86]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


## The architecture (from the paper https://arxiv.org/pdf/1409.1556.pdf)

Notes:
- conv3 = 3x3 convolutional layer
  - stride = 1
  - pad = 1
- maxpool 2x2
  - stride = 2
- weight decay (L2 reg) = 5e-4
- Dropout on first two FC layers
  - ratio = 0.5
- LR = 1e-2
  - decreased by factor of ten on plateau
- batch size = 256

The architecture:
- Input: 224 x 224 RGB (32x32 for CIFAR100  or can resize CIFAR images to 224x224)
- Block 1
  - conv3-64
  - conv3-64
  - maxpool
- block 2
  - conv3-128
  - conv3-128
  - maxpool
- block 3
  - conv3-256
  - conv3-256
  - conv3-256
  - maxpool
- block 4
  - conv3-512
  - conv3-512
  - conv3-512
  - maxpool
- block 5
  - conv3-512
  - conv3-512
  - conv3-512
  - maxpool
- fully connected - 4096
- fully connected - 4096
- fully connected - 1000 (1000 is for ImageNet, we would need 100 for CIFAR)
- softmax



In [87]:
# Define the model
class VGG16(nn.Module):
  def __init__(self, expansion_coef):
    super(VGG16, self).__init__()
    self.name = 'VGG-16'
    self.expansion_coef = expansion_coef

    self.layers = nn.Sequential(
        # Block 1 shape: 224x224
        nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),

        # Block 2 shape: 112x112
        nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),

        # Block 3 shape: 56x56
        nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),

        # Block 4 shape: 28x28
        nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),

        # Block 5 shape: 14x14
        nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),

        # Block 6 shape: 7x7
        nn.Flatten(),
        nn.Linear(512* self.expansion_coef * self.expansion_coef,4096),
        nn.ReLU(),
        nn.Dropout(p=0.5),
        nn.Linear(4096,4096),
        nn.ReLU(),
        nn.Dropout(p=0.5),
        nn.Linear(4096, 100)
    )


  def forward(self, x):
    x = self.layers(x)
    return x

In [133]:
def initialize_weights(layer):
  if isinstance(layer, nn.Conv2d):
    nn.init.xavier_uniform_(layer.weight)
    layer.bias.data.fill_(0.01)

In [88]:
# Training loop
def train(dataloader, batch_size, model, loss_fn, optimizer):
  num_batches = len(dataloader)
  size = num_batches * batch_size
  model.train()
  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    # Compute prediction error
    pred = model(X)
    loss = loss_fn(pred, y)

    # Backprop
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100 == 0:
      loss, current = loss.item(), batch * len(X)
      print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [89]:
# Test function
def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()
  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
  return test_loss

In [90]:
# Validation function
def validation(dataloader, batch_size, model, loss_fn):
  num_batches = len(dataloader)
  size = num_batches * batch_size
  model.eval()
  val_loss, num_correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      val_loss += loss_fn(pred, y).item()
      num_correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  val_loss /= num_batches
  accuracy = num_correct / size
  print(f"Validation Error: \n Validation accuracy: {(100 * accuracy):>0.1f}%, Validation loss: {val_loss:>8f} \n")
  return val_loss

In [91]:
def training_loop(n_epochs, model, train_data, valid_data, batch_size,
                  loss_function, optimizer, scheduler=None,
                  early_stopping=False, patience=10):
  current_epoch = 0
  best_epoch = 0
  best_loss = float('inf')
  patience_counter = 0

  for e in range(n_epochs):
    print(f"\nEpoch {e+1}\n----------------------------")
    # Iterate epoch counter
    current_epoch += 1

    train(train_data, batch_size, model, loss_function, optimizer)
    val_loss = validation(valid_data, batch_size, model, loss_function)

    # Iterate scheduler (this is set up for ReduceLROnPlateau)
    if scheduler is not None:
      scheduler.step(val_loss)

    # If early_stopping check test_loss
    if early_stopping:
      # case: test loss beats the current best loss
      if val_loss < best_loss:
        # store loss
        best_loss = val_loss

        # reset patience counter
        patience_counter = 0

        # store model and epoch number
        print("Storing new best model.")
        best_model_state_dict = copy.deepcopy(model.state_dict)
        best_epoch = current_epoch

      # Case: patience limit not yet reached => iterate patience counter
      elif patience_counter < patience - 1:
        patience_counter += 1
        print(f"Patience count: {patience_counter}")

      # Case: patience limit reached
      else:
        print("Finished due to early stopping.")
        print(f"Saving best model: {model.name}_epoch-{best_epoch:03d}")
        torch.save(best_model_state_dict, f'{model.name}_epoch-{best_epoch:03d}')
        break

  # If we get here, we did not stop early - save best model
  if early_stopping:
    print(f"Saving best model: {model.name}_epoch-{best_epoch:03d}")
    torch.save(best_model_state_dict, f'{model.name}_epoch-{best_epoch:03d}')

# Training

In [163]:
n_epochs = 3

In [164]:
model = VGG16(expansion_coef).to(device)
num_param = sum(p.numel() for p in model.parameters())
num_param_trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)

In [165]:
# Apply initialization
model.apply(initialize_weights)

VGG16(
  (layers): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU()
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU()
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU()
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU()
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU()
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (17): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), pad

In [166]:
# Loss function
loss_fn = nn.CrossEntropyLoss()

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=2e-4)
#optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)

# Scheduler
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, mode='min', patience=5)

In [167]:
training_loop(n_epochs, model, train_dataloader, valid_dataloader, batch_size,
              loss_fn, optimizer, early_stopping=True, patience=10)


Epoch 1
----------------------------
loss: 4.605896 [    0/40000]
loss: 4.613282 [10000/40000]
loss: 4.505758 [20000/40000]
loss: 4.431888 [30000/40000]
Validation Error: 
 Validation accuracy: 2.9%, Validation loss: 4.246157 

Storing new best model.

Epoch 2
----------------------------
loss: 4.128386 [    0/40000]
loss: 4.395105 [10000/40000]
loss: 3.985777 [20000/40000]
loss: 4.077597 [30000/40000]
Validation Error: 
 Validation accuracy: 5.3%, Validation loss: 4.000291 

Storing new best model.

Epoch 3
----------------------------
loss: 4.065510 [    0/40000]
loss: 4.002787 [10000/40000]
loss: 3.850573 [20000/40000]
loss: 3.979860 [30000/40000]
Validation Error: 
 Validation accuracy: 8.1%, Validation loss: 3.826275 

Storing new best model.
Saving best model: VGG-16_epoch-003
